In [1]:
import pandas as pd
from sqlalchemy import create_engine
import pyodbc
import numpy as np
import datetime

In [96]:
server = 'tcp:mycmvserver.database.windows.net' 
database = 'cmvdb_uat' 
username = 'ecervantes@cmvlatam.com' 
password = 'Columbia87/' 


connection = pyodbc.connect('Driver={ODBC Driver 17 for SQL Server};Server=tcp:mycmvserver.database.windows.net,1433;Database=cmvdb_uat;Uid='+username+';Pwd='+password+';Encrypt=yes;TrustServerCertificate=no;Connection Timeout=30;Authentication=ActiveDirectoryInteractive')
cursor = connection.cursor()

In [97]:
monexpp = 619.11

monexbid = 0.00
monexbidpp = 619.00

monexask = 115000.00
monexaskpp = 619.14

In [101]:


##Blueprint   be carefull###

query = "SELECT TOP (100) * FROM [dbo].[fxblotter.t]"
# Read the SQL query into a DataFrame
fxblotter = pd.read_sql(query, connection, index_col='tradeid', parse_dates=True)
##fxblotter[::-1]
fxblotter.sort_index(inplace=True)
## filtro para fecha en T o dia de consulta ### 
##tradedate = '2021-06-16'
##fxblottert = fxblotter.loc[fxblotter['tradedate'] == tradedate] ## se debe ajustar la fecha al inicio de dia##
##fxblottert = fxblotter[fxblotter['tradedate'].dt.strftime('%Y-%m-%d') == '2021-06-16' ]

##fxblottert = fxblotter.groupby(["tradedate"]).get_group(('2021-06-11'))
##fxblottert = fxblotter.query(["tradedate"])

fxblotter['tradedate'] = pd.to_datetime(fxblotter['tradedate'])
fxblotter['valuedate'] = pd.to_datetime(fxblotter['valuedate'])
fxblottert = fxblotter.loc[fxblotter['tradedate'] == '2021-06-17']


tradedate = "2021-06-11" ## this is not needed##

##Falta hacer filtros para deals perdidos y deals delete con posicion inverssa
bidtotalcliente = fxblottert.loc[fxblotter['comccyid'] == 1, 'montocom'].sum()
asktotalcliente = fxblottert.loc[fxblotter['venccyid'] == 1, 'montoven'].sum()


###FX Posicion Import fxblotter.t as a DF to make calculations###
bidtotal = (fxblottert.loc[fxblottert['comccyid'] == 1, 'montocom'].sum() + monexbid)
asktotal = (fxblottert.loc[fxblottert['venccyid'] == 1, 'montoven'].sum() + monexask)

###fxnetposclientes = bidtotal - asktotal ### USD Moneda base ###
fxnetpos = (bidtotal) - (asktotal) ### USD Moneda base ###
fxnetposclientes = (bidtotalcliente) - (asktotalcliente) ### USD Moneda base ###
## Weighted average 
bidclientpp = fxblottert.loc[fxblottert['comccyid'] == 1, 'montoven'].sum() / bidtotalcliente
askclientpp = fxblottert.loc[fxblottert['venccyid'] == 1, 'montocom'].sum() / asktotalcliente

bidpp = (fxblottert.loc[fxblottert['comccyid'] == 1, 'montoven'].sum() +(monexbid*monexbidpp)) / bidtotal
askpp = (fxblottert.loc[fxblottert['venccyid'] == 1, 'montocom'].sum() +(monexask*monexaskpp)) / asktotal
##PnL##
if fxnetpos > 0:   
    coveredpnl =  asktotal * (askpp - bidpp)
    uncoveredpnl = (bidtotal - asktotal) * (monexpp - bidpp)  
else :
    coveredpnl =  bidtotal * (askpp - bidpp)
    uncoveredpnl = (asktotal - bidtotal) * (askpp - monexpp)
        ###uncoveredpnl

        ##if fxnetpos > 0 :   
        ##else :   
    
totalpnl = coveredpnl + uncoveredpnl

##print(f"daily total pnl is: {totalpnl}")
##print(f"FX net position:  {fxnetpos}")

print1 ="FX net position:  {:,} USD"
print(print1.format(fxnetpos))
print2 ="FX clients net position:  {:,} USD"
print(print2.format(fxnetposclientes))

print3 ="Bid client amount:  {:,} @ {}"
print(print3.format(bidtotalcliente, bidclientpp))

print4 ="Ask client amount:  {:,} @ {}"
print(print4.format(asktotalcliente, askclientpp))

print5 ="Bid total amount:  {:,} @ {}"
print(print5.format(bidtotal, bidpp))

print6 ="Ask total amount:  {:,} @ {}"
print(print6.format(asktotal, askpp))

print7 ="daily total pnl is:  {:,} crc"
print(print7.format(totalpnl))

##fxblottert.head(100)

FX net position:  39.470000000001164 USD
FX clients net position:  115,039.47 USD
Bid client amount:  130,840.79000000001 @ 616.4038060302142
Ask client amount:  15,801.32 @ 617.8816706452373
Bid total amount:  130,840.79000000001 @ 616.4038060302142
Ask total amount:  130,801.32 @ 618.9879888062292
daily total pnl is:  338,121.3317000043 crc


In [86]:
##Deal Management Blotter###
qtrades = fxblottert['sideid'].value_counts()
qestado = fxblottert['estadoid'].value_counts()
print(qtrades)
print(qestado)


1    14
2     3
Name: sideid, dtype: int64
1    17
Name: estadoid, dtype: int64


## FX Posicion

<mark>Import fxblotter.t as a DF to make calculations</mark>

<mark>netpos = \[monto total de ventas (USD moneda base)\]  -  \[monto total de compra (USD moneda base)\]</mark>

## Stats

asktotal y askpp 

bidtotal y bidpp

## PnL 

PnL =  min \[total offer ,total bid \] \* \[ pp de offer  -  pp de bid\]  + if \[netpos \>0 , (bidtotal - offertotal) \* (monex.pp - bidpp), (offertotal - bidtotal) \* (offerpp - monexpp) \]

## DMB

compras, ventas, cantidad por 'estado', monex compra venta

## <span style="font-size: 14px;">definir compras y ventas de monex como variables float insert</span>

<span style="font-size: 14px;"><br></span>